![Use case 1](../../static/img/summarization_use_case_1.png)

In [8]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

In [10]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models.gigachat import GigaChat
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# loader = TextLoader("assets/arxiv_paper.pdf")
# docs = loader.load()

loader = PyPDFLoader("assets/arxiv_paper.pdf")
docs = loader.load()

# loader = WebBaseLoader("https://habr.com/ru/companies/sberdevices/articles/794773/")
# docs = loader.load()

split_docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=1000).split_documents(
    docs
)
print(f"Parts count: {len(split_docs)}")

Parts count: 7


Далее с помощью `GigaChat`` выполняется суммаризация каждой части (фаза map). В конце все суммаризированные части объединяются в одну, после чего выполняется суммарная суммаризация.

In [11]:
giga = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
chain = load_summarize_chain(giga, chain_type="map_reduce")
res = chain.run(split_docs)

print("\n\n===")
print(res)



===
This paper presents a method for estimating in-grasp torque from visuotactile sensors using the concept of the Tactile Dipole Moment. The method does not rely on deep learning or sensor-specific modeling, but instead takes inspiration from electromagnetism to analyze the vector field produced from 2D marker displacements. Experiments with a real robot demonstrate the accuracy of the method in measuring tilt torques during a USB stick insertion task. The method is applicable to other visuotactile sensing hardware and grasped object shapes. The authors open source the implementation of their algorithm and experimental apparatus design files.


### Погружаемся глубже
 
**Кастомизация** 

* Вы можете самостоятельно настроить промпты для каждой фазы map-reduce.

**Реальные примеры использования**

* См. [this blog post](https://blog.langchain.dev/llms-to-improve-documentation/) кейс по анализу взаимодействия с пользователем (вопросы по документации LangChain)!  
* Данном посте [repo](https://github.com/mendableai/QA_clustering) представлена кластеризация как средство суммаризации.
* Это открывает третий путь, помимо подходов «stuff» или «map-reduce», который стоит рассмотреть.

![Image description](../../static/img/summarization_use_case_3.png)

## Подход 3. Refine
 
[Refine](/docs/modules/chains/document/refine) подход похож на mad-reduce:

> Цепочка refine создает ответ, перебирая входные документы и итеративно обновляя свой ответ. В цикле происходит суммаризация каждого документа с добавлением полученной на предыдущем шаге суммаризации предыдущих документов.

Вы можете легко запустить эту цепочку, использовав параметр `chain_type="refine"`.

In [10]:
chain = load_summarize_chain(giga, chain_type="refine")
chain.run(split_docs)

'GigaChain - это форк открытой библиотеки LangChain на Python, который был создан для упрощения жизни разработчика. Он содержит множество компонентов, позволяющих работать с промптами, объединять вызовы к большим языковым моделям в цепочки, загружать данные из разных источников и сохранять обработанные ответы языковой модели. GigaChain отличается от LangChain тем, что он переведен на русский язык, адаптирован к экосистеме русскоязычных языковых моделей и, конечно же, к GigaChat. Фреймворк совместим со сторонним сервисом LangSmith - платформой для разработчиков, которая позволяет отлаживать, тестировать, оценивать и отслеживать цепочки, построенные на любой платформе LLM. GigaChain содержит большое количество инструментов и примеров, которые можно использовать в качестве основы своего проекта или просто изучать для расширения кругозора в области возможных кейсов применений LLM.\n\nRAG (Retrieval Augmented Generation) - это подход к обогащению вопроса пользователя дополнительными данными

It's also possible to supply a prompt and return intermediate steps.

In [11]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=giga,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Giga generation stopped with reason: length


In [12]:
print(result["output_text"])

Le nostre opportunità sono di raffinare il sommario esistente con un po' più di contesto sottostante.


In [13]:
print("\n\n".join(result["intermediate_steps"][:3]))

GigaChain - это форк открытой библиотеки LangChain на Python, который был создан для упрощения жизни разработчика. Он содержит множество компонентов, позволяющих работать с промптами, объединять вызовы к большим языковым моделям в цепочки, загружать данные из разных источников и сохранять обработанные ответы языковой модели. GigaChain отличается от LangChain тем, что он переведен на русский язык, адаптирован к экосистеме русскоязычных языковых моделей и, конечно же, к GigaChat. Фреймворк совместим со сторонним сервисом LangSmith - платформой для разработчиков, которая позволяет отлаживать, тестировать, оценивать и отслеживать цепочки, построенные на любой платформе LLM.

La nostra opportunità è di raffinare il sommario esistente con un po' più di contesto sottostante.

------------
In questi casi, uno dei metodi è RAG (Retrieval Augmented Generation), che spiegherò in seguito.

Risposte alle domande dai documenti (RAG)
L'idea di RAG è arricchire la domanda del utente con informazioni a

В случае ниже статья не превышает 1000 токенов, поэтому мы суммаризуем её методом Stuff

В случае ниже, у нас документы превышают 1000 токенов, поэтому мы используем Map-Reduce